In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    num

In [4]:
pip install BeautifulSoup4

     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lxml

     |████████████████████████████████| 5.8MB 24.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


#### Use beautifulSoup to scrape data from wiki

In [6]:
from bs4 import BeautifulSoup
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
 

#### Clean the table

In [7]:
df3 = df.copy()
df2 = []
for i in range(0,len(df)):
   if df3.iloc[i,1] != "Not assigned" and df3.iloc[i,2] != "Not assigned" and df3.iloc[i,1] != df3.iloc[i-1,1]:
     df2.append(df3.iloc[i,:]) 
   elif df3.iloc[i,1] != "Not assigned" and df3.iloc[i,2] != "Not assigned" and df3.iloc[i,1] == df3.iloc[i-1,1] and df3.iloc[i,0] == df3.iloc[i-1,0]:    
    df2[len(df2)-1][2] = df3.iloc[i-1,2] + "," + df3.iloc[i,2]
   elif df3.iloc[i,1] != "Not assigned" and df3.iloc[i,2] == "Not assigned":
     df2.append(df3.iloc[i,:])
     df2[len(df2)-1][2] = df2[len(df2)-1][1]
df2 = pd.DataFrame(df2)  

In [8]:
df2.shape

(88, 3)

In [9]:
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park


In [10]:
Location = pd.read_csv("http://cocl.us/Geospatial_data")

In [11]:
Location.columns = ['Postcode', 'Latitude','Longitude']

In [12]:
Location.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
result = pd.merge(df2, Location, on='Postcode')

In [23]:
result = result.iloc[:,1:]

In [25]:
result

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,Downtown Toronto,Harbourfront,43.654260,-79.360636
2,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
3,Queen's Park,Queen's Park,43.662301,-79.389494
4,Downtown Toronto,Queen's Park,43.667856,-79.532242
5,Scarborough,"Rouge,Malvern",43.806686,-79.194353
6,North York,Don Mills North,43.745906,-79.352188
7,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
8,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
9,North York,Glencairn,43.709577,-79.445073


#### Create a map of New York with neighborhoods superimposed on top.¶

In [26]:
address = 'Downtown Toronto	, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


### create a function to get location for all the neighborhoods in Toronto 

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
Toronto_data = result[result['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

In [29]:
Toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Define Foursquare Credentials and Version

In [30]:
CLIENT_ID = 'MDKTZ51MSTHC0B4LJJKK4HGKKRJF0IAGHZ4Y5WPOH3FHTHGN' # your Foursquare ID
CLIENT_SECRET = 'VJRMO15QZNBGKLP12YY4YXFVVSMPLIHJKFYQPFY0F3TNK30C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MDKTZ51MSTHC0B4LJJKK4HGKKRJF0IAGHZ4Y5WPOH3FHTHGN
CLIENT_SECRET:VJRMO15QZNBGKLP12YY4YXFVVSMPLIHJKFYQPFY0F3TNK30C


### Get Downtown Toronto nearby venues

In [31]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )



Harbourfront
Queen's Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
King,Richmond
Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Grange Park,Kensington Market
Railway Lands,South Niagara
Rosedale
First Canadian Place,Underground city
Church and Wellesley


In [32]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1066, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


check how many venues were returned for each neighborhood

In [33]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
Central Bay Street,84,84,84,84,84,84
Church and Wellesley,83,83,83,83,83,83
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100
"First Canadian Place,Underground city",43,43,43,43,43,43
"Grange Park,Kensington Market",94,94,94,94,94,94
"Harbord,University of Toronto",38,38,38,38,38,38
Harbourfront,50,50,50,50,50,50


how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 199 uniques categories.


### Analyze Each Neighborhood

In [35]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.columns

Index(['Yoga Studio', 'Afghan Restaurant', 'Airport', 'Airport Food Court',
       'Airport Gate', 'Airport Lounge', 'Airport Service', 'Airport Terminal',
       'American Restaurant', 'Antique Shop',
       ...
       'Theme Restaurant', 'Thrift / Vintage Store', 'Toy / Game Store',
       'Trail', 'Train Station', 'Vegetarian / Vegan Restaurant',
       'Video Game Store', 'Vietnamese Restaurant', 'Wine Bar', 'Wings Joint'],
      dtype='object', length=199)

In [36]:
Toronto_onehot.shape

(1066, 199)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.018182,0.000000,0.00,0.018182,0.018182,0.036364,0.000000,0.000000,0.00,0.018182,0.018182,0.000000,0.036364,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.018182,0.036364,0.00,0.000000,0.036364,0.000000,0.00,0.00,0.018182,0.036364,0.090909,0.000000,0.000000,0.00,0.00,0.018182,0.000000,0.018182,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.036364,0.000000,0.00,0.000000,0.018182,0.000000,0.000000,0.000000,0.00,0.018182,0.018182,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.018182,0.018182,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.018182,0.018182,0.018182,0.018182,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.018182,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.018182,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.018182,0.00,0.000000,0.0000

In [38]:
Toronto_grouped.shape

(15, 199)

### Print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1       Bakery  0.04
2   Steakhouse  0.04
3         Café  0.04
4  Cheese Shop  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1      Sandwich Place  0.05
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4      Ice Cream Shop  0.04


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.06
1     Sushi Restaurant  0.06
2              Gay Bar  0.05
3  Japanese Restaurant  0.05
4           Restaurant  0.04


----Commerce Court,Victoria Hotel----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.07
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


----Design Exchange,Toronto Dominion Centre----
         venue  freq
0  Coffee Shop  0.13
1         Café  0.08
2        Hotel  0.07
3   Restaurant  0.05
4          Bar  0.04


----First Canadian Place,Underground city----
              

### create the new dataframe and display the top 10 venues for each neighborhood.

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [240]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
1,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Café,Gym / Fitness Center,Japanese Restaurant,Bubble Tea Shop,Spa
2,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Pub,Men's Store,Bubble Tea Shop,Mediterranean Restaurant,Gastropub
3,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Steakhouse,Italian Restaurant,Seafood Restaurant,Deli / Bodega
4,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,American Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,Gastropub


### Run k-means to cluster the neighborhood into 5 clusters.

In [241]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 0, 0, 0, 0, 3, 4, 0], dtype=int32)

In [242]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_data = Toronto_data.rename(columns={"Neighbour": "Neighborhood"})
toronto_merged = Toronto_data
#Toronto_data['Neighbourhood'] = Toronto_data['Neighbourhood'].astype(str)
#neighborhoods_venues_sorted = neighborhoods_venues_sorted.reset_index()
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#neighborhoods_venues_sorted = neighborhoods_venues_sorted.rename(columns={"Neighborhood": "Neighbour"})
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#toronto_merged = pd.merge(Toronto_data, neighborhoods_venues_sorted, how='left', on=['Neighborhood'])
toronto_merged = toronto_merged.drop(toronto_merged.index[1])
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,4.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Café,Theater,Restaurant,Beer Store
2,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Bakery,Italian Restaurant,Bookstore,Pizza Place
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Café,Coffee Shop,Restaurant,Hotel,Clothing Store,Italian Restaurant,Breakfast Spot,Beer Bar,American Restaurant,Diner
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,Coffee Shop,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
5,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Café,Gym / Fitness Center,Japanese Restaurant,Bubble Tea Shop,Spa
6,Downtown Toronto,"King,Richmond",43.650571,-79.384568,0.0,Coffee Shop,Café,Thai Restaurant,Steakhouse,Restaurant,Bakery,Bar,Asian Restaurant,Sushi Restaurant,Burger Joint
7,Downtown Toronto,"Toronto Islands,Union Station",43.640816,-79.381752,0.0,Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Restaurant,Italian Restaurant,Scenic Lookout,Sporting Goods Shop
8,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,0.0,Coffee Shop,Café,Hotel,Restaurant,Bar,American Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,Gastropub
9,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,0.0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Steakhouse,Italian Restaurant,Seafood Restaurant,Deli / Bodega
10,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,3.0,Café,Sandwich Place,Italian Restaurant,Bar,Japanese Restaurant,Bakery,Restaurant,Theater,Bookstore,Dessert Shop


In [246]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters